In [ ]:
import re 
import datetime
import pandas as pd

import urllib.request as urllib_request
from urllib.error import URLError, HTTPError

from fiis_scraping.utils.helpers import *
from fiis_scraping.utils.functions import *
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [ ]:
URL = 'https://fiis.com.br/lista-de-fundos-imobiliarios/'
soup = create_bsoup_object(URL)

In [ ]:
tickers = get_tickers(soup)

In [ ]:
dataset = create_fiis_dataframe(tickers)

In [ ]:
COLUMNS_LIST = ['ultimo_rendimento',
                'valor_patrimonial_por_cota',
                'numero_cotas',
                'numero_cotistas',
                'cotacao',
                'min_52_weeks',
                'max_52_weeks']
dataset = convert_string_to_float(dataset, COLUMNS_LIST)
dataset = convert_percentages_to_float(dataset, ['dividend_yield', 'return_12_months'])
dataset = break_tipo_FII(dataset)
dataset['registro_CVM'] = dataset['registro_CVM'].apply(lambda x: str(convert_to_datetime(x)))
#dataset["p_vp"] = (dataset['cotacao'] / dataset['valor_patrimonial_por_cota']).replace(np.inf, np.nan)
dataset['date'] = str(datetime.date.today() - datetime.timedelta(days=1))
dataset.head()

In [ ]:
URL = 'https://fiis.com.br/ifix/'
df_ifix = create_ifix_dataframe(URL)
COLUMNS_LIST = ['peso']
df_ifix = convert_percentages_to_float(df_ifix, COLUMNS_LIST)
df_ifix['peso'] = df_ifix['peso'].round(4)
df_ifix['date'] = str(datetime.date.today())
df_ifix.head()

In [ ]:
df_ifix.info()

## DynamoDB

### Sending the fiis dataframe

In [ ]:
import boto3
import json
from decimal import Decimal

In [ ]:
#ticker_data_dict = dataset.to_dict(orient='records')
ticker_data_dict = dataset.to_dict(orient='records')

In [ ]:
dynamodb_resource = boto3.resource("dynamodb")

In [ ]:
batch_write(dynamodb_resource, "fiis", ticker_data_dict)

In [ ]:
table = dynamodb_resource.Table("fiis")
response = table.scan()
df = pd.DataFrame(response['Items'])
df.shape

In [ ]:
df.date.value_counts(dropna=False)

### Sending the ifix dataframe

In [ ]:
dataset = df_ifix

In [ ]:
dynamodb_resource = boto3.resource("dynamodb")
ifix_data_dict = dataset.to_dict(orient='records')
batch_write(dynamodb_resource, "ifix", ifix_data_dict)